# Dummy Provider Example and High Volume Robustness Testing

This notebook has two purposes: 

- Demonstrate the dummy feedback function provider which behaves like the
  huggingface provider except it does not actually perform any network calls and
  just produces constant results. It can be used to prototype feedback function
  wiring for your apps before invoking potentially slow (to run/to load)
  feedback functions.

- Test out high-volume record and feedback computation. To this end, we use the
  custom app which is dummy in a sense that it produces useless answers without
  making any API calls but otherwise behaves similarly to real apps, and the
  dummy feedback function provider.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

In [ ]:
from concurrent.futures import as_completed

from tqdm.auto import tqdm
from trulens.core import Feedback
from trulens.core import Tru
from trulens.core import TruCustomApp
from trulens.core.utils.threading import TP
from trulens.dashboard import run_dashboard
from trulens.providers.huggingface.provider import Dummy

from examples.dev.dummy_app.app import DummyApp

tp = TP()

d_error = Dummy(
    loading_prob=0.0,
    freeze_prob=0.0,  # we expect requests to have their own timeouts so freeze should never happen
    error_prob=1.0,
    overloaded_prob=0.0,
    rpm=1000,
    alloc=0,  # how much fake data to allocate during requests
    delay=10.0,
)

d_noerror = Dummy(
    loading_prob=0.0,
    freeze_prob=0.0,  # we expect requests to have their own timeouts so freeze should never happen
    error_prob=0.0,
    overloaded_prob=0.0,
    rpm=1000,
    alloc=0,  # how much fake data to allocate during requests
    delay=10.0,
)

tru = Tru()

# tru.reset_database()

run_dashboard(tru, force=True, _dev=Path().cwd().parent.parent.resolve())

In [ ]:
tru.get_records_and_feedback(limit=10)[0]

In [ ]:
f_dummy1 = Feedback(d_error.language_match).on_input_output()

f_dummy2 = Feedback(
    d_noerror.positive_sentiment, name="output sentiment"
).on_output()

# Create custom app:
ca = DummyApp(delay=0.0, alloc=0)

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
    feedbacks=[f_dummy1, f_dummy2],
    #    feedback_mode=FeedbackMode.DEFERRED
)

In [ ]:
with ta as recorder:
    res = ca.respond_to_query("hello there")
res

In [ ]:
ta.wait_for_feedback_results(feedback_timeout=5)

In [ ]:
# Sequential app invocation.

if True:
    for i in tqdm(range(2), desc="invoking app"):
        with ta as recorder:
            ca.respond_to_query(f"hello {i}")

        rec = recorder.get()
        assert rec is not None

In [ ]:
# ta.wait_for_feedback_results()

In [ ]:
# Control retries in deferred evaluator.
# tru.RETRY_FAILED_SECONDS = 60
# tru.RETRY_RUNNING_SECONDS = 5
tru.start_evaluator(restart=True)

In [ ]:
# Parallel feedback evaluation.

futures = []
num_tests = 10000
good = 0
bad = 0


def test_feedback(msg):
    return msg, d_error.positive_sentiment(msg)


for i in tqdm(range(num_tests), desc="starting feedback task"):
    futures.append(tp.submit(test_feedback, msg="good"))

prog = tqdm(as_completed(futures), total=num_tests)

for f in prog:
    try:
        res = f.result()
        good += 1

        assert res[0] == "good"

        prog.set_description_str(f"{good} / {bad}")
    except Exception:
        bad += 1
        prog.set_description_str(f"{good} / {bad}")

In [ ]:
# Parallel app invocation.


def run_query(q):
    with ta as recorder:
        ca.respond_to_query(q)

    rec = recorder.get()
    assert rec is not None

    return f"run_query {q} result"


for i in tqdm(range(100), desc="starting app task"):
    print(tp.completed_tasks, end="\r")
    tp.submit(run_query, q=f"hello {i}")